In [ ]:
pip install foolbox

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random
import foolbox as fb
import torch.optim as optim


trainMode = True
constrain = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from utils import *
from network import *
from training import *
from multiClassHinge import *

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [ ]:
# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
batch_size = 128


trainset =  torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

a = 0.5
M = 1

In [ ]:
class Normalisation(torch.nn.Module):
    def __init__(self,means=(0.485, 0.456, 0.406), stds=(0.229, 0.224, 0.225)): #cifar100
        super().__init__()
        assert len(means) == len(stds)
        self.means = means
        self.stds = stds

    def forward(self, x):
        return (x - torch.tensor(self.means, device=x.device).view(1, len(self.means), 1, 1)) / torch.tensor(self.stds, device=x.device).view(1, len(self.means), 1, 1)



transform_test_rob = transforms.Compose([
    transforms.ToTensor()
])
batch_size = 1024
testset_rob = torchvision.datasets.CIFAR100(root='./data', train=False,
                                      download=True, transform=transform_test_rob)
testloader_rob = torch.utils.data.DataLoader(testset_rob, batch_size=batch_size,
                                        shuffle=False, num_workers=2)
images, labels = next(iter(testloader_rob))
images, labels = images.to(device), labels.to(device)

In [ ]:
marginList = [0.07,0.15,0.3]
epsilons = [0.0, 8/255, 16/255, 36/255, 0.3, 0.5, 0.6, 0.8, 1.0]
saveRobAcc = np.zeros((len(marginList),len(epsilons)))
it = 0
for num,margin in enumerate(marginList):

  print(f"\n\n Now we are working with margin = {margin}\n\n")

  net = Network(3,32,64,128,4,4,4,S = 2, a = a, M=M)
  net.to(device);
  pretrained_dict = torch.load(f"CertifiedLip_margin_{margin}_cifar10.pt",map_location=device)
  net.load_state_dict(pretrained_dict, strict=False)
  for k in range(3):
    net.seq[k].u.data  = torch.rand(net.seq[k].nlayers)

  criterion = multiClassHingeLoss(margin = margin)
  lr = 1e-4

  optimizer = optim.SGD(net.parameters(),lr=lr,momentum=0.9)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

  model_save_name = f'CertifiedLip_margin_{margin}.pt'
  path = F"{model_save_name}"

  if trainMode:
    loss = -1
    while loss<0:
      print(f"Training with learning rate {lr}")
      loss = train(net, margin, criterion, optimizer, scheduler, trainloader, testloader, device, epochs=100, reg=True, a=a, constrain = constrain, gamma=1)
      if loss<0:
        net = Network(3,32,64,128,4,4,4,S=2,a=a,M=M)
        net.to(device);
        lr/=5
        optimizer = optim.SGD(net.parameters(),lr=lr,momentum=0.9)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

    torch.save(net.state_dict(), path)
  else:
    net.load_state_dict(torch.load(path), strict=False)

  model = nn.Sequential(Normalisation(),net).eval()
  fmodel = fb.PyTorchModel(model, bounds=(0, 1))

  acc = fb.utils.accuracy(fmodel, images, labels)

  attack = fb.attacks.L2PGD(steps=10)
  _, advs, success = attack(fmodel, images, labels, epsilons=epsilons)
  robust_accuracy = torch.mean((1-1.*success),axis=1)
  saveRobAcc[num] = robust_accuracy.detach().cpu().numpy()

  it += 1
  destination = f"updateMargin_{margin}.txt"
  np.savetxt(destination, saveRobAcc.reshape(-1,1))

  fig = plt.figure(figsize=(20,10))
  plt.plot(epsilons,robust_accuracy.detach().cpu().numpy(),'r-*',label="Experimental")
  plt.xlabel(r"$\varepsilon$",fontsize=20)
  plt.ylabel("Robust accuracy",fontsize=20)
  plt.xticks(fontsize=20)
  plt.yticks(fontsize=20)
  plt.legend(fontsize=20,loc=1)
  plt.title(f"L2 robustness. Trained with margin = {margin}. Test accuracy = {round(acc * 100,2)}%",fontsize=20);

  plt.savefig(f'L2margin_{margin}.png')

fig = plt.figure(figsize=(20,10))

for i in range(len(marginList)):
  plt.plot(epsilons,saveRobAcc[i],'-*',label=f"Margin = {marginList[i]}")

plt.xlabel(r"$\varepsilon$",fontsize=20)
plt.ylabel("Robust accuracy",fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20,loc=1)
plt.title(f"L2 robustness comparison CIFAR100",fontsize=20);

plt.savefig(f'L2RobustnessComparison.png')